# Look ma' - no loops!

This is a small tutorial containing Python recipes for loopless programming.

## What is loopless programming?

As the name suggests, loopless is about avoiding `for`- and `while` loops.
Less loops in your code, often simplifies the code and lets us focus on *what* we are trying to achieve, rather than *how*.

Note that throughout this tutorial we will use the term `sequence` interchangably with `iterable`, even though technically a [Sequence](https://docs.python.org/3/glossary.html#term-sequence) in Python is different from an [Iterable](https://docs.python.org/3/glossary.html#term-iterable).

This little tutorial will provide tiny recipes sorted by topic.
But first, let's define a little function for providing more interesting output.

In [ ]:
from IPython.display import display, Markdown


# this little function prints markdown instead of raw text
def print_markdown(markdown: str) -> None:
    display(Markdown(markdown))

## Combining sequences

One common task in programming is combining sequences of data. This use case has two varieties that we are going to take a look at next.

### Simple pairing

For pairing two sequences of equal length, we can use the built-in [`zip()`](https://docs.python.org/3/library/functions.html#zip)
function.

**Recipe 1**:

> `combined = zip(seq1, seq2, ...)`

**Example**:

In [ ]:
a = [1, 2, 3, 4]
b = ['solo', 'duet', 'trio', 'quartette']

c = zip(a, b)
print(list(c))

### Merging sequences of unequal lengths

What if we have two sequences of different lengths? In this case we have a few ways of generating a combined result.

#### Option 1: Combine using the shortest sequence

This is what the `zip()` function des by default.

**Recipe 2.1**:

> `shortest_combination = zip(seq1, seq2, ...)`

**Example:**

In [ ]:
a = [1, 2, 3, 4, 5]
b = ['solo', 'duet', 'trio', 'quartette']

c = zip(a, b)
print(list(c))

#### Option 2: Padding

A second way of combining sequences of different lengths is to pair the extra elements in the longest  sequences with some fixed value. This is often referred to as *padding* and can be achieved using [`itertools.zip_longest`](https://docs.python.org/3/library/itertools.html#itertools.zip_longest).

**Recipe 2.2**:

> `padded_combined_seq = zip_longest(seq1, seq2, ..., fillvalue=padding_value)`

**Example**:

In [ ]:
from itertools import zip_longest

a = [1, 2, 3, 4, 5, 23]
b = ['solo', 'duet', 'trio', 'quartette']

c = zip_longest(a, b, fillvalue='many')
print(list(c))

#### Option 3: Repeat the shorter sequence

The function [`itertools.cycle`](https://docs.python.org/3/library/itertools.html#itertools.cycle) allows us to
restart an iterator after it returned its last element.

**Recipe 2.3**:

> `combined = zip(longer_seq, cycle(shorter_seq), ...)`

**or**

> `combined = zip(cycle(shorter_seq), longer_seq, ...)`

**Example**:

In [ ]:
from itertools import cycle

a = ['red', 'green', 'blue', 'yellow']
b = ['TRIANGLE', 'CIRCLE']

c = zip(a, cycle(b))
print_markdown('➡'.join(f'<span style="color:{col}">{txt}</span>' for col, txt in c))

#### Option 4: Repeat a single value

We can also pair input sequences with a single value using [`itertools.repeat`](https://docs.python.org/3/library/itertools.html#itertools.repeat).

**Recipe 2.4**:

> `combined = zip(seq1, ..., repeat(some_value))`

**Example**:

In [ ]:
from itertools import repeat

a = [10, 12, 15, 18]

c = zip(a, repeat('SAMPLE TEXT'))
print_markdown('•'.join(f'<span style="font-size:{sz}pt">{txt}</span>' for sz, txt in c))

### All combinations of multiple sequences

Oftentimes we want to generate all combinations of multiple sequences.
Take a card game, for example. We have suits and card values and want to generate a list of all available cards in the game. Such combination is called a [*Cartesian product*](https://en.wikipedia.org/wiki/Cartesian_product) and can be generated using [`itertools.product`](https://docs.python.org/3/library/itertools.html#itertools.product).

**Recipe 3**:

> `all_combinations = product(seq1, seq2, ...)`

**Example**:

In [ ]:
from itertools import product

suites = ['♠', '♣', '♥', '♦']
values = ['ace', 'king', 'queen', 'jack', '10', '9', '8', '7']

deck = product(suites, values)
print(list(deck))

## Flatten nested sequences

A [common question](https://stackoverflow.com/questions/25674169/how-does-the-list-comprehension-to-flatten-a-python-list-work) is how to turn a nested sequence into a flat sequence.

Since we're in the *loopless* camp, we don't want no list comprehensions😉. Instead, we can use [`itertools.chain`](https://docs.python.org/3/library/itertools.html#itertools.chain) and a [`starred expression`](https://docs.python.org/3/reference/expressions.html#grammar-token-python-grammar-starred_item).

**Recipe 4**:

> `flattened = chain(*nested)`

**Example**:

In [ ]:
from itertools import chain

def f(x):
    return x + 10

def g(x):
    return x * 10

# some operation that returns tuples of (f(x), g(x))
data = [1, 23, 4, 17, 9, 2, 49]
result = chain(*((f(x), g(x)) for x in data))
print(list(result))

### Multiple nesting levels

The `chain`-method can in principle be applied to more deeply nested sequences:

In [ ]:
lines = [((1, 2, 3), (4, 5, 6)), ((7, 8, 9), (10, 11, 12))]

coords = chain(*chain(*lines))

print(list(coords))

It might, however, be more readable and cleaner to do the flattening explicitly instead:

In [ ]:
lines = [((1, 2, 3), (4, 5, 6)), ((7, 8, 9), (10, 11, 12))]

points = chain(*lines)      # list of pairs of vecs -> sequence of vecs
coords = chain(*points)     # sequence of vecs -> sequence of individual coords

print(list(coords))

## Splitting nested sequences

Sometimes it's useful to split a nested sequence into multiple sequences, each containing elements of the input sequence.

We can use a combination of `zip()` and a *starred expresision* to do this looplessly:

**Recipe 5a**:

> `*zip(*nested)`

In contexts where a *starred expression* cannot be used, we can replace it with `map()` to generate a tuple of outputs instead:

**Recipe 5b**:

> `seq1, seq2, ... = map(iter|list, zip(*nested))`

**Example**:

In [ ]:
parameters_by_name = {
    'x': 1.0,
    'y': -7.2,
    'z': 0.0
}

names, values = map(iter, zip(*parameters_by_name.items()))
print(list(names))
print(list(values))

## Transforming data

Data sequences can be transformed by extracting the transformation step into a function and using `map()` to apply the function to each element:

**Recipe 6**:

> results = map(transformation_fn, seq_of_args, ...)

**Example 1 - Single input**:

Let's transform a sequence of integers by applying $ f: x \to x^2 $

In [ ]:
def f(x):
    return x ** 2

squares = map(f, range(1, 21))
print(list(squares))

**Example 2 - Multiple inputs**:

Functions with multiple inputs work by passing multiple arguments to `map()`.
Say we have $f: \Bbb{N}^2 \to \Bbb{R}, (x, y) \to \frac{x^2}{y}$

In [ ]:
def f(x, y):
    return x ** 2 / y

results = map(f, range(10), range(1, 11))
print(list(results))

**Example 3 - Tuple input:**

Since the use-case of the previous example is so common and data may already be tuples,
`itertools.starmap()` unpacks tuples for us if the input is just a single iterable.

In [ ]:
from itertools import starmap


styles = {
    'font-size': '18pt',
    'color': 'steelblue',
    'font-weight': 'bold',
    'text-decoration': 'underline'
}

def css_prop(property, value):
    return f'{property}:{value}'

props = starmap(css_prop, styles.items())
span = f'<span style="{";".join(props)}">Hello, world!</span>'
print_markdown(span)

## Filtering data

Selecting elements from a sequence based on some boolean function ("criterion" or "predicate") can be done using `filter()` function. `itertools.filterfalse()` adds
the ability to filter for the case when the predicate function returns false.

**Recipe 7:**

> matching_elements = filter(predicate, sequence)

**-or-**

> nonmatching_elements = filterfalse(predicate, sequence)

**Example**:

In [ ]:
def is_multiple_of_4(x):
    return not x % 4

numbers = range(1, 21)
multiples_of_4 = filter(is_multiple_of_4, numbers)
print(list(multiples_of_4))

## Replacing nested loops

Using the tools we've looked at so far, we can now procede to replace nested `for`-loops with loopless versions.

For a motivating example, let's consider a use case where we want to schedule the play-offs between two divisions of an e-sports league.

We might start with a looped implementation similar to this:

In [ ]:
division_a = ['The Squirrels', 'Hometown Girls', 'Geeks United']
division_b = ['Scooter Gang', 'Slow Pokes']
game_modes = ['Best-of-Three', 'Lucky-Shot']


def schedule_game(team_a, team_b, game_mode):
    return f'scheduling {game_mode} match between {team_a} and {team_b}'

schedule = []
for team_a in division_a:
    for team_b in division_b:
        for mode in game_modes:
            match = schedule_game(team_a, team_b, mode)
            schedule.append(match)

print('\n'.join(schedule))

Looking at the loops, we can see that we have a case of *Cartesian product*, which we already know how to handle:

In [ ]:
from itertools import product

all_combinations = list(product(division_a, division_b, game_modes))
print(all_combinations)

Finally, we can *transform* these lists to our final result by using `starmap()` and `schedule_game()`:

In [ ]:
from itertools import starmap

schedule = starmap(schedule_game, all_combinations)
print('\n'.join(schedule))

Putting it all together, we can replace the nested `for`-loop version with this loopless one-liner:

In [ ]:
from itertools import product, starmap

schedule = starmap(schedule_game, product(division_a, division_b, game_modes))
print('\n'.join(schedule))

### General recipe for replacing nested loops

To summarise how to replace nested `for`-loops that iterate over multiple inputs:

**Recipe 8:**

> `result = starmap(loop_body_fn, product(outermost_inputs, middle_inputs, ...))`

### Refactoring the loop body

The first step to convert existing nested `for`-loops is to move the loop-body into a separate function. Pass all variables used in the loop into that function starting with non-loop variables.

Anything the loop body calculates and stores somewhere should be part of the return value instead, e.g.

In [ ]:
import random

colours = ['green', 'lime', 'yellow', 'orange']
font_sizes = [10, 14]
sample_texts = ['Foo', 'Bar']

# non-loop variable
delim = random.choice('!?;:')

markdown_text = []
for colour in colours:
    for font_size in font_sizes:
        for text in sample_texts:
            # -- loop body --
            style = f'font-size:{font_size}pt; color:{colour}'
            sample = f'<span style="{style}">{text}{delim}</span>'
            markdown_text.append(sample)  # calculated output
            # ---------------

In [ ]:
# step 1: move loop body into a separate function:

def loop_body_fn(delim, colour, font_size, text):
    style = f'font-size:{font_size}pt; color:{colour}'
    sample = f'<span style="{style}">{text}{delim}</span>'
    return sample

Next we can apply the recipe. We can use `list()` to trigger the evaluation:

In [ ]:
from itertools import product, starmap

_ = list(starmap(loop_body_fn, product(colours, font_sizes, sample_texts)))

Unfortunately, this doesn't work. We somehow need to pass our non-loop variable `delim` to `loop_body_fn()` somehow. We could use the `repeat()` recipe to pass a sequence that always returns the value of `delim` as the first parameter:

In [ ]:
# don't uncomment the lines below and run this 😉
# _ = list(starmap(loop_body_fn, product(repeat(delim), colours, font_sizes, sample_texts)))

Hm. We'd rather not run this, because since `repeat()` will never stop returning elements, this will cause problems. Of course we could switch the order of the parameters or pass the number of times to `repeat()`, but that's not a good general solution.

Instead, we can use `functools.partial()` to get a function that will have the first (or any, really) argument fixed to whatever we want:

In [ ]:
from itertools import product, starmap
from functools import partial

fixed_loop_body = partial(loop_body_fn, delim)
markdown = starmap(fixed_loop_body, product(colours, font_sizes, sample_texts))
print_markdown(''.join(markdown))

## Advanced: Splitting sequences based on a predicate

So far, we've dealt with separating sequences into sequences of the same length.
What if we wanted to split an input sequence based on an arbitrary predicate?

In [ ]:
from itertools import filterfalse

candidates = iter(range(1, 13))


def is_multiple_of_4(x):
    return not x % 4


multiples_of_4 = filter(is_multiple_of_4, candidates)
other_values = filterfalse(is_multiple_of_4, candidates)
print(list(multiples_of_4))
print(list(other_values))

What happened here, why is our second sequence empty? Note how `candidates` uses `iter()` to turn the range into an iteration. Since we exhaust the iterator with the first call to `filter()` already, the subsequent call to `filterfalse()` will get an empty sequence.

Luckily, there's the `itertools.tee()` function, which can return multiple independent iterators from the given one:

In [ ]:
from itertools import filterfalse, tee

candidates = iter(range(1, 13))


def is_multiple_of_4(x):
    print(f'🔎{x}')
    return not x % 4

# get an independent iterator for both filter passes
it1, it2 = tee(candidates)

multiples_of_4 = filter(is_multiple_of_4, it1)
other_values = filterfalse(is_multiple_of_4, it2)
print(list(multiples_of_4))
print(list(other_values))

That worked! Great, but we also noticed that it still looks at the input twice.
This might be very costly if the predicate is a complex function, or worse straight up impossible if our input iterator cannot be restarted (e.g. when we are processing a non-seekable stream of data).

We can evaluate both, the input and the predicate in one go and return the value along with its evaluated predicate:

In [ ]:
candidates = iter(range(1, 18))

def pred(x):
    return not x % 4

evaluated = map(lambda x: (x, pred(x)), candidates)
print(list(evaluated))

Now we can filter the evaluated results. This still means iterating twice, but only over an already evaluated set of data.

In [ ]:
from itertools import filterfalse, tee

candidates = iter(range(1, 13))

def pred(x):
    return not x % 4

evaluated = map(lambda x: (x, pred(x)), candidates)
# remember that we need two independent iterators
iter_matches, iter_others = tee(evaluated)
matches = filter(lambda x: x[1], iter_matches)
others = filterfalse(lambda x: x[1], iter_others)
print(list(matches))
print(list(others))

Finally we need to transform the filtered results by discarding the second tuple member.

We can put this into a `partition()` function that can be reused whenever the functionality is needed:

In [2]:
from functools import partial
from itertools import filterfalse, tee
from typing import Callable, Iterable, TypeVar, Union


_T = TypeVar('_T')


def partition(
    predicate: Callable[[_T], bool], iterable: Iterable[_T]
) -> tuple[Iterable[_T], Iterable[_T]]:
    """Return a 2-tuple of iterables from an input iterable based on a
    predicate.

    Args:
        predicate (Callable): Predicate that takes an element of the iterable
            and returns whether element matches some criterion

        iterable (iter): Iterable to be partitioned in matching and non-matching
            elements

    Returns:
        (tuple[iter, iter]) The first iterable yields all matching elements in the
            input. The second iterable yields all non-matching elements of the
            input.
    """
    def _get(n: int, tuple_: tuple[_T, bool]) -> Union[_T, bool]:
        return tuple_[n]

    first, second = partial(_get, 0), partial(_get, 1)
    evaluations = map(lambda x: (x, predicate(x)), iterable)
    iter_matches, iter_others = tee(evaluations)
    return (
        map(first, filter(second, iter_matches)),
        map(first, filterfalse(second, iter_others))
    )

Let's try our previous example using the `partition()` function: 

In [ ]:
candidates = iter(range(1, 14))


def is_multiple_of_4(x):
    print(f'🔎{x}')
    return not x % 4


multiples_of_4, other_values = partition(is_multiple_of_4, candidates)

print(list(multiples_of_4))
print(list(other_values))

As we can see, every value in the input is only looked at once.
If you're interested in more tools like `partition()`, there's
the [**more-itertools**](https://github.com/more-itertools/more-itertools) libary,
which also contains a `partition()` function (though it disagrees on the order of
the returned iterators) among many other useful tools for loopless programming.

This is also the point where our little journey ends.